# JR95-250129: qPCR Analysis Report - Full Details

## TODO ADD STYLING TO NOTEBOOK OR HTML FILE OUTPUT

## Summary

### Description

What the experiment is but more importantly emphasize the purpose.

Briefly describe the experimental design (SSCT).

### Results

What are the main results save details for later, overall good, overall bad.

### Conclusion

What the MINE conclusions at this point in the analysis.
What do I think my next steps are?
More importantly, what feedback am I looking for and seeking from the audience to 
pay particular attention to?

## Table of Content

## Code & Analysis

### Setup

In [1]:
import pandas as pd
from bokeh.plotting import figure, show

from dctap.libs.pandas import set_defaultoptions, display, displaydf_full
from dctap.qpcr.constants import QPCRPATHS
from dctap.qpcr.utils.core import *

In [2]:
# Set pandas settings
set_defaultoptions(pd, supresscopywarning=None)

### Reading in the data

#### Collecting and annotating the data

In [3]:
# Read and annotate data from different plates and combine them
experiment_id = "JR95-250129"
plate_ids = ["JR95-250129-plate1", "JR95-250129-plate2", "JR95-250129-plate3"]

dfs = []
for plate_id in plate_ids:
    dfs.append(get_plate_data(experiment_id, plate_id))

df = pd.concat(dfs)
df.reset_index(inplace=True, drop=True)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id
0,JR95-250129,A01,P6_D0-2DD_SD0.4_TR1_0hr,CER1,28.067467,JR95-250129-plate1
1,JR95-250129,A02,P6_D0-2DD_SD0.4_TR1_0hr,CER1,27.777831,JR95-250129-plate1
2,JR95-250129,A03,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.799317,JR95-250129-plate1
3,JR95-250129,A04,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.671907,JR95-250129-plate1
4,JR95-250129,A05,P6_D0-2DD_SD0.46_TR1_0hr,CER1,28.448518,JR95-250129-plate1
...,...,...,...,...,...,...
759,JR95-250129,N12,P12_D2-6DD_SD0.4_TR2_48hr,NANOG,23.667099,JR95-250129-plate3
760,JR95-250129,N13,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.529697,JR95-250129-plate3
761,JR95-250129,N14,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.609485,JR95-250129-plate3
762,JR95-250129,N15,P12_D2-6DD_SD0.5_TR2_48hr,NANOG,24.069101,JR95-250129-plate3


#### Demultiplex Sample labels and conditions

This will be helpful in dynamically assigning controls samples,
biological replicate samples, condition samples, and so on
for calculating foldchange based on the deltadeltacqMethod.

In [4]:
df_samples = get_sample_metadata(cast(pd.Series, df.Sample), sep="_")
with displaydf_full():
  display(df_samples)

,Sample,0,1,2,3,4
0,P6_D0-2DD_SD0.4_TR1_0hr,P6,D0-2DD,SD0.4,TR1,0hr
1,P6_D0-2DD_SD0.4_TR2_0hr,P6,D0-2DD,SD0.4,TR2,0hr
2,P6_D0-2DD_SD0.46_TR1_0hr,P6,D0-2DD,SD0.46,TR1,0hr
3,P6_D0-2DD_SD0.46_TR2_0hr,P6,D0-2DD,SD0.46,TR2,0hr
4,P12_D0-2DD_SD0.4_TR1_0hr,P12,D0-2DD,SD0.4,TR1,0hr
5,P12_D0-2DD_SD0.4_TR2_0hr,P12,D0-2DD,SD0.4,TR2,0hr
6,P12_D0-2DD_SD0.5_TR1_0hr,P12,D0-2DD,SD0.5,TR1,0hr
7,P12_D0-2DD_SD0.5_TR2_0hr,P12,D0-2DD,SD0.5,TR2,0hr
8,P6_D2-4DD_SD0.4_TR1_24hr,P6,D2-4DD,SD0.4,TR1,24hr
9,P6_D2-4DD_SD0.4_TR2_24hr,P6,D2-4DD,SD0.4,TR2,24hr


#### Adding helpful meta-data

In [5]:
conditions = ["bio_reps", "ctrl_calibrator", "cond_sd", "cond_chirtime"]
df = set_conditions(
    df,
    df_samples,
    conditions=conditions,
    merge_cols=["0124", "012", "02", "4"],
)

display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime
0,JR95-250129,A01,P6_D0-2DD_SD0.4_TR1_0hr,CER1,28.067467,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
1,JR95-250129,A02,P6_D0-2DD_SD0.4_TR1_0hr,CER1,27.777831,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
2,JR95-250129,A03,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.799317,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
3,JR95-250129,A04,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.671907,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
4,JR95-250129,A05,P6_D0-2DD_SD0.46_TR1_0hr,CER1,28.448518,JR95-250129-plate1,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr
...,...,...,...,...,...,...,...,...,...,...
759,JR95-250129,N12,P12_D2-6DD_SD0.4_TR2_48hr,NANOG,23.667099,JR95-250129-plate3,P12_D2-6DD_SD0.4_48hr,P12_D2-6DD_SD0.4,P12_SD0.4,48hr
760,JR95-250129,N13,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.529697,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
761,JR95-250129,N14,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.609485,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
762,JR95-250129,N15,P12_D2-6DD_SD0.5_TR2_48hr,NANOG,24.069101,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr


#### Bulk Calculate deltaCq expression

In [6]:
df1 = get_deltaCq_expression_bulkdata(
    df,
    ref_primer="GAPDH",
    test_primers=get_primers(df),
    drop_customcols=conditions,
)
display(df)

,experiment_id,Well,Sample,Primer,Cq,plate_id,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime
0,JR95-250129,A01,P6_D0-2DD_SD0.4_TR1_0hr,CER1,28.067467,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
1,JR95-250129,A02,P6_D0-2DD_SD0.4_TR1_0hr,CER1,27.777831,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
2,JR95-250129,A03,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.799317,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
3,JR95-250129,A04,P6_D0-2DD_SD0.4_TR2_0hr,CER1,27.671907,JR95-250129-plate1,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr
4,JR95-250129,A05,P6_D0-2DD_SD0.46_TR1_0hr,CER1,28.448518,JR95-250129-plate1,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr
...,...,...,...,...,...,...,...,...,...,...
759,JR95-250129,N12,P12_D2-6DD_SD0.4_TR2_48hr,NANOG,23.667099,JR95-250129-plate3,P12_D2-6DD_SD0.4_48hr,P12_D2-6DD_SD0.4,P12_SD0.4,48hr
760,JR95-250129,N13,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.529697,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
761,JR95-250129,N14,P12_D2-6DD_SD0.5_TR1_48hr,NANOG,23.609485,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr
762,JR95-250129,N15,P12_D2-6DD_SD0.5_TR2_48hr,NANOG,24.069101,JR95-250129-plate3,P12_D2-6DD_SD0.5_48hr,P12_D2-6DD_SD0.5,P12_SD0.5,48hr


#### Get Calibrators from control samples

In [7]:
df2 = get_deltaCq_stats(df1, biorep_col="bio_reps")
df_calibrators = get_calibrators(
    df2,
    ctrl_col="ctrl_calibrator",
    condition_col="cond_sd",
    assign_ctrl_samples=[
        "P6_D0-2DD_SD0.4",
        "P6_D0-2DD_SD0.46",
        "P12_D0-2DD_SD0.4",
        "P12_D0-2DD_SD0.5",
    ],
    assign_cond_group=[
        "P6_SD0.4",
        "P6_SD0.46",
        "P12_SD0.4",
        "P12_SD0.5",
    ],
)
with displaydf_full():
  display(df_calibrators)

,deltaCq_CER1vGAPDH,deltaCq_LHX1vGAPDH,deltaCq_BETA-ACTINvGAPDH,deltaCq_OCT4vGAPDH,deltaCq_NANOGvGAPDH
P6_SD0.4,7.855217,14.488657,-2.035303,0.041693,4.227435
P6_SD0.46,7.444340,14.698866,-2.275825,-0.125105,4.355624
P12_SD0.4,7.882852,15.102660,-1.847673,0.298518,4.404494
P12_SD0.5,7.632371,13.818526,-2.032231,0.010748,4.017147


#### Get foldchange by deltadeltaCq Method

In [8]:
df3 = get_deltadeltaCqMethod_foldchange(
    df1, df_calibrators, biorep_col="bio_reps", condition_col="cond_sd"
)

with displaydf_full():
  display(df3)

,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime,2^(deltadeltaCq_CER1vGAPDH)_mean,2^(deltadeltaCq_CER1vGAPDH)_std,2^(deltadeltaCq_CER1vGAPDH)_ste,2^(deltadeltaCq_LHX1vGAPDH)_mean,2^(deltadeltaCq_LHX1vGAPDH)_std,2^(deltadeltaCq_LHX1vGAPDH)_ste,2^(deltadeltaCq_BETA-ACTINvGAPDH)_mean,2^(deltadeltaCq_BETA-ACTINvGAPDH)_std,2^(deltadeltaCq_BETA-ACTINvGAPDH)_ste,2^(deltadeltaCq_OCT4vGAPDH)_mean,2^(deltadeltaCq_OCT4vGAPDH)_std,2^(deltadeltaCq_OCT4vGAPDH)_ste,2^(deltadeltaCq_NANOGvGAPDH)_mean,2^(deltadeltaCq_NANOGvGAPDH)_std,2^(deltadeltaCq_NANOGvGAPDH)_ste
0,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr,1.017531,0.265966,0.188066,1.031320,0.356709,0.252231,1.002503,0.100113,0.070791,1.000046,0.013502,0.009548,1.000780,0.055853,0.039494
1,P6_D0-2DD_SD0.46_0hr,P6_D0-2DD_SD0.46,P6_SD0.46,0hr,1.012614,0.225331,0.159333,1.017598,0.266477,0.188428,1.000415,0.040728,0.028799,1.000494,0.044458,0.031437,1.005863,0.153361,0.108443
2,P12_D0-2DD_SD0.4_0hr,P12_D0-2DD_SD0.4,P12_SD0.4,0hr,1.023981,0.311565,0.220309,1.015384,0.249021,0.176084,1.000285,0.033742,0.023859,1.002713,0.104247,0.073714,1.024243,0.313286,0.221526
3,P12_D0-2DD_SD0.5_0hr,P12_D0-2DD_SD0.5,P12_SD0.5,0hr,1.005362,0.146651,0.103698,1.017853,0.268418,0.189800,1.010662,0.207063,0.146416,1.001742,0.083503,0.059046,1.005228,0.144799,0.102388
4,P6_D2-4DD_SD0.4_24hr,P6_D2-4DD_SD0.4,P6_SD0.4,24hr,54.522519,1.565384,1.106894,360.208403,52.625506,37.211852,0.957362,0.240722,0.170216,0.899489,0.101247,0.071593,0.892414,0.177386,0.125431
5,P6_D2-4DD_SD0.46_24hr,P6_D2-4DD_SD0.46,P6_SD0.46,24hr,125.754725,6.897661,4.877383,1103.610465,84.395832,59.676865,1.030118,0.059657,0.042184,0.602145,0.087419,0.061814,0.867039,0.083943,0.059357
6,P12_D2-4DD_SD0.4_24hr,P12_D2-4DD_SD0.4,P12_SD0.4,24hr,19.202386,6.505699,4.600224,182.793880,101.490170,71.764388,0.924078,0.360071,0.254609,0.586849,0.124578,0.088090,0.813957,0.339534,0.240087
7,P12_D2-4DD_SD0.5_24hr,P12_D2-4DD_SD0.5,P12_SD0.5,24hr,3.320770,0.761661,0.538576,9.230736,3.617988,2.558304,0.922106,0.031585,0.022334,0.515593,0.014338,0.010138,0.445685,0.011594,0.008198
8,P6_D2-5DD_SD0.4_32hr,P6_D2-5DD_SD0.4,P6_SD0.4,32hr,74.349174,14.814887,10.475707,1000.260100,360.005321,254.562204,0.879196,0.019769,0.013979,0.472775,0.044287,0.031315,0.523896,0.060381,0.042696
9,P6_D2-5DD_SD0.46_32hr,P6_D2-5DD_SD0.46,P6_SD0.46,32hr,55.067341,7.596206,5.371329,1183.321323,189.495830,133.993787,0.980220,0.157231,0.111179,0.523761,0.061286,0.043336,0.640169,0.098682,0.069779


#### Creating Graphs

##### WTC11 P6 Seeding Density 0.4M OCT4 

In [11]:

df = df3.copy()
pattern = r"^P12_.*_SD0\.5_.*"
df = df[df.bio_reps.str.match(pattern, na=False)]

,bio_reps,ctrl_calibrator,cond_sd,cond_chirtime,2^(deltadeltaCq_CER1vGAPDH)_mean,2^(deltadeltaCq_CER1vGAPDH)_std,2^(deltadeltaCq_CER1vGAPDH)_ste,2^(deltadeltaCq_LHX1vGAPDH)_mean,2^(deltadeltaCq_LHX1vGAPDH)_std,2^(deltadeltaCq_LHX1vGAPDH)_ste,2^(deltadeltaCq_BETA-ACTINvGAPDH)_mean,2^(deltadeltaCq_BETA-ACTINvGAPDH)_std,2^(deltadeltaCq_BETA-ACTINvGAPDH)_ste,2^(deltadeltaCq_OCT4vGAPDH)_mean,2^(deltadeltaCq_OCT4vGAPDH)_std,2^(deltadeltaCq_OCT4vGAPDH)_ste,2^(deltadeltaCq_NANOGvGAPDH)_mean,2^(deltadeltaCq_NANOGvGAPDH)_std,2^(deltadeltaCq_NANOGvGAPDH)_ste
0,P6_D0-2DD_SD0.4_0hr,P6_D0-2DD_SD0.4,P6_SD0.4,0hr,1.017531,0.265966,0.188066,1.031320,0.356709,0.252231,1.002503,0.100113,0.070791,1.000046,0.013502,0.009548,1.000780,0.055853,0.039494
4,P6_D2-4DD_SD0.4_24hr,P6_D2-4DD_SD0.4,P6_SD0.4,24hr,54.522519,1.565384,1.106894,360.208403,52.625506,37.211852,0.957362,0.240722,0.170216,0.899489,0.101247,0.071593,0.892414,0.177386,0.125431
8,P6_D2-5DD_SD0.4_32hr,P6_D2-5DD_SD0.4,P6_SD0.4,32hr,74.349174,14.814887,10.475707,1000.260100,360.005321,254.562204,0.879196,0.019769,0.013979,0.472775,0.044287,0.031315,0.523896,0.060381,0.042696
12,P6_D2-6DD_SD0.4_48hr,P6_D2-6DD_SD0.4,P6_SD0.4,48hr,27.234153,10.942521,7.737531,909.512286,357.336834,252.675298,0.495906,0.042830,0.030286,0.234551,0.121452,0.085880,0.242481,0.077294,0.054655
